# TP04 — Differential Kinematics & Jacobian Analysis

**GEII3 — Industrie 4.0: Robotique | Lab Session 4**

**Duration**: 2 sessions × 4 hours  
**Prerequisites**: TP02 (FK), TP03 (IK), linear algebra, calculus

---

## Learning Objectives

By the end of this lab (8 hours total), you should be able to:

1. Understand the **Jacobian matrix** and its role in robot kinematics

2. Compute Jacobians using **three methods**:
   - Analytical differentiation
   - Geometric approach (screw theory)
   - Numerical approximation

3. Analyze **singularities** and their effects:
   - Identify singular configurations
   - Compute manipulability measures
   - Visualize manipulability ellipsoid

4. Apply Jacobian for **velocity kinematics**:
   - Forward velocity problem
   - Inverse velocity problem
   - Cartesian velocity control

5. Understand **force-torque relationships**:
   - Static force analysis
   - Virtual work principle
   - Joint torques from end-effector forces

6. Implement **practical applications**:
   - Singularity avoidance
   - Resolved motion rate control
   - Force control basics

---

## The Jacobian Matrix - Foundation

### Definition

The **Jacobian matrix** $J(\mathbf{q})$ relates joint velocities to end-effector velocities:

$$\mathbf{v} = J(\mathbf{q}) \cdot \dot{\mathbf{q}}$$

where:
- $\mathbf{v} \in \mathbb{R}^6$: end-effector twist (3 linear + 3 angular velocities)
- $\dot{\mathbf{q}} \in \mathbb{R}^n$: joint velocities
- $J \in \mathbb{R}^{6 \times n}$: Jacobian matrix

### Structure

$$J = \begin{bmatrix}
J_v \\
J_\omega
\end{bmatrix} = \begin{bmatrix}
\frac{\partial x}{\partial q_1} & \cdots & \frac{\partial x}{\partial q_n} \\
\vdots & \ddots & \vdots \\
\frac{\partial \omega_z}{\partial q_1} & \cdots & \frac{\partial \omega_z}{\partial q_n}
\end{bmatrix}$$

- $J_v$ (3×n): Linear velocity part
- $J_\omega$ (3×n): Angular velocity part

---

## Why is the Jacobian Important?

### 1. Velocity Control
Control end-effector velocity by commanding joint velocities:
$$\dot{\mathbf{q}} = J^{-1} \mathbf{v}_d$$

### 2. Singularity Analysis
When $\det(J) = 0$ or $\text{rank}(J) < 6$:
- Robot loses degrees of freedom
- Some directions become unreachable
- Infinite joint velocities may be required

### 3. Force/Torque Mapping
Via principle of virtual work:
$$\boldsymbol{\tau} = J^T \mathbf{F}$$

### 4. Manipulability
Measure of robot's ability to move in all directions:
$$w = \sqrt{\det(J J^T)}$$

---

## Lab Organization

### **Session 1** (4 hours)
- **Part I**: Theory & setup (20 min)
- **Exercise 1**: 2R planar - analytical Jacobian (40 min)
- **Exercise 2**: Geometric Jacobian approach (50 min)
- **Exercise 3**: Numerical Jacobian & validation (40 min)
- **Exercise 4**: Singularity analysis - 2R and 3R (70 min)

### **Session 2** (4 hours)
- **Exercise 5**: Manipulability analysis & ellipsoid (60 min)
- **Exercise 6**: Velocity kinematics & resolved motion control (70 min)
- **Exercise 7**: Force analysis & statics (50 min)
- **Exercise 8**: Advanced topics & applications (60 min)

---

# Part I: Theoretical Background

## Three Methods to Compute the Jacobian

### 1. Analytical Method (Differentiation)

**Approach**: Differentiate forward kinematics equations

Given FK: $\mathbf{p} = f(\mathbf{q})$

$$J_{ij} = \frac{\partial p_i}{\partial q_j}$$

**Advantages**: Exact, symbolic expressions  
**Disadvantages**: Complex for many DOF, representation-dependent

---

### 2. Geometric Method (Screw Theory)

**Approach**: Use axis of motion and position vectors

For **revolute joint i**:
$$J_i = \begin{bmatrix}
\mathbf{z}_{i-1} \times (\mathbf{p}_n - \mathbf{p}_{i-1}) \\
\mathbf{z}_{i-1}
\end{bmatrix}$$

For **prismatic joint i**:
$$J_i = \begin{bmatrix}
\mathbf{z}_{i-1} \\
\mathbf{0}
\end{bmatrix}$$

where:
- $\mathbf{z}_{i-1}$: axis of joint i
- $\mathbf{p}_n$: end-effector position
- $\mathbf{p}_{i-1}$: position of joint i

**Advantages**: Geometric intuition, works for any robot  
**Disadvantages**: Requires FK computation first

---

### 3. Numerical Method (Finite Differences)

**Approach**: Approximate derivatives numerically

$$J_{ij} \approx \frac{p_i(q_j + \epsilon) - p_i(q_j)}{\epsilon}$$

**Advantages**: Easy to implement, no analytical work  
**Disadvantages**: Numerical errors, computational cost

---

## Setup and Installation

In [ ]:
# Uncomment to install:
# !pip install -q roboticstoolbox-python spatialmath-python matplotlib scipy

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Ellipse
from math import cos, sin, pi, sqrt, atan2
from scipy.linalg import svd, det, pinv

try:
    import roboticstoolbox as rtb
    from spatialmath import SE3, SO3
    RTB_AVAILABLE = True
    print(f"✓ RTB version: {rtb.__version__}")
except ImportError:
    RTB_AVAILABLE = False
    print("⚠ RTB not available")

np.set_printoptions(precision=4, suppress=True, linewidth=100)
print("✓ Libraries loaded\n")

## Helper Functions

In [ ]:
# ============================================================================
# FORWARD KINEMATICS (from previous TPs)
# ============================================================================

def dh_standard(theta, d, a, alpha):
    """Standard DH transformation."""
    ct, st = cos(theta), sin(theta)
    ca, sa = cos(alpha), sin(alpha)
    return np.array([
        [ct, -st*ca,  st*sa, a*ct],
        [st,  ct*ca, -ct*sa, a*st],
        [0,      sa,     ca,    d],
        [0,       0,      0,    1]
    ])

def fk_dh(dh_params, q, convention='standard'):
    """Forward kinematics using DH."""
    T = np.eye(4)
    T_list = [T.copy()]
    
    for i, (theta_off, d_off, a, alpha, jtype) in enumerate(dh_params):
        theta = (theta_off + q[i]) if jtype == 'r' else theta_off
        d = d_off if jtype == 'r' else (d_off + q[i])
        
        T_i = dh_standard(theta, d, a, alpha)
        T = T @ T_i
        T_list.append(T.copy())
    
    return T, T_list

def extract_position(T):
    """Extract position from transformation."""
    return T[:3, 3]

def extract_rotation(T):
    """Extract rotation from transformation."""
    return T[:3, :3]


# ============================================================================
# JACOBIAN COMPUTATION METHODS
# ============================================================================

def jacobian_numerical(dh_params, q, epsilon=1e-6):
    """
    Numerical Jacobian using finite differences.
    Returns 6×n Jacobian matrix.
    """
    n = len(q)
    J = np.zeros((6, n))
    
    T_0, _ = fk_dh(dh_params, q)
    p_0 = extract_position(T_0)
    R_0 = extract_rotation(T_0)
    
    for i in range(n):
        q_delta = q.copy()
        q_delta[i] += epsilon
        
        T_delta, _ = fk_dh(dh_params, q_delta)
        p_delta = extract_position(T_delta)
        R_delta = extract_rotation(T_delta)
        
        # Linear velocity (position derivative)
        J[:3, i] = (p_delta - p_0) / epsilon
        
        # Angular velocity (rotation derivative approximation)
        dR = (R_delta - R_0) / epsilon
        # Skew-symmetric part
        J[3, i] = (dR[2,1] - dR[1,2]) / 2
        J[4, i] = (dR[0,2] - dR[2,0]) / 2
        J[5, i] = (dR[1,0] - dR[0,1]) / 2
    
    return J


def jacobian_geometric(dh_params, q):
    """
    Geometric Jacobian using screw theory.
    Each column is computed from joint axis and positions.
    """
    n = len(q)
    J = np.zeros((6, n))
    
    # Get all transformation matrices
    T_end, T_list = fk_dh(dh_params, q)
    p_n = extract_position(T_end)  # End-effector position
    
    for i in range(n):
        T_i = T_list[i]  # Transform to joint i
        p_i = extract_position(T_i)  # Joint i position
        R_i = extract_rotation(T_i)  # Joint i orientation
        
        # Joint axis in world frame (z-axis after transform)
        z_i = R_i[:, 2]
        
        jtype = dh_params[i][4]
        
        if jtype == 'r':  # Revolute
            # Linear velocity: z_i × (p_n - p_i)
            J[:3, i] = np.cross(z_i, p_n - p_i)
            # Angular velocity: z_i
            J[3:, i] = z_i
        else:  # Prismatic
            # Linear velocity: z_i
            J[:3, i] = z_i
            # Angular velocity: 0
            J[3:, i] = 0
    
    return J


# ============================================================================
# JACOBIAN ANALYSIS
# ============================================================================

def manipulability(J):
    """
    Compute manipulability measure (Yoshikawa).
    w = sqrt(det(J * J^T))
    """
    JJT = J @ J.T
    det_JJT = det(JJT)
    return sqrt(abs(det_JJT))


def condition_number(J):
    """
    Compute condition number of Jacobian.
    κ = σ_max / σ_min (ratio of largest to smallest singular value)
    """
    U, s, Vt = svd(J)
    return s[0] / s[-1] if s[-1] > 1e-10 else np.inf


def is_singular(J, threshold=1e-3):
    """
    Check if configuration is singular.
    """
    m = manipulability(J)
    return m < threshold


def singularity_distance(J):
    """
    Distance to nearest singularity (minimum singular value).
    """
    U, s, Vt = svd(J)
    return s[-1]


# ============================================================================
# VELOCITY KINEMATICS
# ============================================================================

def forward_velocity(J, q_dot):
    """
    Compute end-effector velocity from joint velocities.
    v = J * q_dot
    """
    return J @ q_dot


def inverse_velocity(J, v_desired, method='pinv', damping=0.01):
    """
    Compute joint velocities for desired end-effector velocity.
    
    Methods:
    - 'pinv': Moore-Penrose pseudo-inverse
    - 'dls': Damped Least Squares
    - 'transpose': Jacobian transpose
    """
    if method == 'pinv':
        J_inv = pinv(J)
    elif method == 'dls':
        n = J.shape[1]
        J_inv = J.T @ np.linalg.inv(J @ J.T + damping**2 * np.eye(6))
    elif method == 'transpose':
        J_inv = J.T
    else:
        raise ValueError(f"Unknown method: {method}")
    
    return J_inv @ v_desired


# ============================================================================
# FORCE/TORQUE ANALYSIS
# ============================================================================

def force_to_torque(J, F):
    """
    Convert end-effector force/moment to joint torques.
    τ = J^T * F
    """
    return J.T @ F


def torque_to_force(J, tau):
    """
    Convert joint torques to end-effector force.
    F = (J^T)^{-1} * τ  (if square and invertible)
    """
    JT_inv = pinv(J.T)
    return JT_inv @ tau


# ============================================================================
# VISUALIZATION
# ============================================================================

def plot_robot_2d(T_list, ax=None, title="Robot Configuration", J=None):
    """Visualize 2D robot with optional Jacobian info."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 10))
    
    x = [T[0, 3] for T in T_list]
    y = [T[1, 3] for T in T_list]
    
    ax.plot(x, y, 'o-', linewidth=2, markersize=8, label='Robot')
    ax.plot(x[0], y[0], 'gs', markersize=12, label='Base')
    ax.plot(x[-1], y[-1], 'r*', markersize=15, label='End-effector')
    
    # Add manipulability info if Jacobian provided
    if J is not None:
        w = manipulability(J)
        singular = is_singular(J)
        status = "SINGULAR" if singular else "OK"
        color = 'red' if singular else 'green'
        ax.text(0.02, 0.98, f"Manipulability: {w:.4f}\nStatus: {status}",
                transform=ax.transAxes, fontsize=11, verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor=color, alpha=0.3))
    
    ax.grid(True, alpha=0.3)
    ax.axis('equal')
    ax.set_xlabel('X (m)', fontsize=11)
    ax.set_ylabel('Y (m)', fontsize=11)
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.legend(loc='best')
    
    return ax


print("✓ Helper functions defined\n")

---

# Exercise 1: 2R Planar Robot - Analytical Jacobian

**Estimated time**: 40 minutes (Session 1)

## Problem Description

2-DOF planar robot:
- $L_1 = 0.4$ m, $L_2 = 0.3$ m

**Forward kinematics**:
$$\begin{aligned}
x &= L_1\cos(q_1) + L_2\cos(q_1+q_2) \\
y &= L_1\sin(q_1) + L_2\sin(q_1+q_2)
\end{aligned}$$

## Task 1.1: Analytical Derivation (20 min)

**Instructions**: Compute Jacobian by differentiating FK equations.

### Your derivation:

$$J = \begin{bmatrix}
\frac{\partial x}{\partial q_1} & \frac{\partial x}{\partial q_2} \\
\frac{\partial y}{\partial q_1} & \frac{\partial y}{\partial q_2}
\end{bmatrix}$$

**Step 1**: Differentiate $x$:
$$\frac{\partial x}{\partial q_1} = -L_1\sin(q_1) - L_2\sin(q_1+q_2)$$
$$\frac{\partial x}{\partial q_2} = -L_2\sin(q_1+q_2)$$

**Step 2**: Differentiate $y$:
$$\frac{\partial y}{\partial q_1} = ?$$
$$\frac{\partial y}{\partial q_2} = ?$$

**Final Jacobian**:
$$J = \begin{bmatrix}
? & ? \\
? & ?
\end{bmatrix}$$

---

## Task 1.2: Implementation (20 min)

In [ ]:
# Robot parameters
L1 = 0.4
L2 = 0.3

dh_2r = [
    (0, 0, L1, 0, 'r'),
    (0, 0, L2, 0, 'r'),
]

def jacobian_2r_analytical(q1, q2, L1, L2):
    """
    Analytical Jacobian for 2R planar robot.
    Returns 2×2 matrix (only position, planar motion).
    """
    # TODO: Implement based on your derivation
    
    # Partial derivatives of x
    dx_dq1 = -L1*sin(q1) - L2*sin(q1 + q2)
    dx_dq2 = -L2*sin(q1 + q2)
    
    # Partial derivatives of y
    dy_dq1 = L1*cos(q1) + L2*cos(q1 + q2)
    dy_dq2 = L2*cos(q1 + q2)
    
    J = np.array([
        [dx_dq1, dx_dq2],
        [dy_dq1, dy_dq2]
    ])
    
    return J


# Test configurations
print("\n" + "="*70)
print("  2R PLANAR - ANALYTICAL JACOBIAN")
print("="*70)

test_configs = [
    ([np.deg2rad(30), np.deg2rad(45)], "Moderate config"),
    ([0, 0], "Stretched (singular)"),
    ([np.deg2rad(90), np.deg2rad(-90)], "Folded"),
    ([np.deg2rad(45), np.deg2rad(90)], "Elbow up"),
]

for q, desc in test_configs:
    print(f"\n[{desc}] q = {np.rad2deg(q).astype(int)}°")
    
    # Analytical Jacobian
    J_ana = jacobian_2r_analytical(q[0], q[1], L1, L2)
    
    print(f"\nJacobian (2×2):")
    print(J_ana)
    
    # Determinant
    det_J = det(J_ana)
    print(f"\ndet(J) = {det_J:.6f}")
    
    if abs(det_J) < 1e-3:
        print("⚠ SINGULAR configuration!")
    else:
        print("✓ Non-singular")
    
    # Manipulability (for 2D: |det(J)|)
    w = abs(det_J)
    print(f"Manipulability: {w:.6f}")